In [1]:
import sys 
sys.path.append('../')
import pandas as pd 
import numpy as np 
import talib
import sqlite3 as sql
import matplotlib.pyplot as plt 
from backtester import * 
from fxcmtoken import my_assets

In [2]:
for i, a in enumerate(my_assets):
    print(i, a)

0 EUR/USD
1 USD/JPY
2 GBP/USD
3 USOil
4 SPX500
5 VOLX
6 XAU/USD
7 Bund
8 BTC/USD


In [13]:
num_periods =  [i for i in range(25, 201, 25)]

for n_period in num_periods:
    conn = sql.connect('../PriceData/PriceData_m5.db')
    ticker = my_assets[8]
    print(ticker)
    data = pd.read_sql(f"SELECT * FROM '{ticker}'", conn, parse_dates=['date'], index_col=['date'])
    for col in ['open', 'close', 'high', 'low']:
        data[col] = data[['bid'+col, 'ask'+col]].mean(axis=1)
    conn.close()
    data

    print(f"Period: {n_period}")

    data['ht'] = talib.HT_TRENDLINE(data.close)
    data['ht_ema'] = talib.EMA(data.ht, n_period)
    data['signals'] = np.where(data.ht > data.ht_ema, 1, -1)

#     data[['close', 'slope_bin']].plot(figsize=(12,6), secondary_y='slope_bin')
#     plt.tight_layout()
#     plt.show()

    data.dropna(axis=0, inplace=True)

    backtest = IterativeBacktester(data=data,
                                   signals=data.signals,
                                   freq='m5')

    backtest.backtest(progress_bar=False)
    # backtest.results()
    print(f"{backtest.return_df.loc['TotalReturn', 'Portfolio']*100:.4f}%")
    
    print("Signals")
    print(backtest.signals.value_counts())

    print("Signals change")
    print(backtest.signals.diff(1).dropna().apply(np.abs).value_counts())
    
    print("-"*50)

BTC/USD
Period: 25
-16.0144%
Signals
-1    5196
 1    4717
Name: signals, dtype: int64
Signals change
0.0    9657
2.0     255
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 50
-3.7398%
Signals
-1    5289
 1    4599
Name: signals, dtype: int64
Signals change
0.0    9714
2.0     173
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 75
-6.9288%
Signals
-1    5339
 1    4524
Name: signals, dtype: int64
Signals change
0.0    9727
2.0     135
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 100
5.5890%
Signals
-1    5419
 1    4419
Name: signals, dtype: int64
Signals change
0.0    9726
2.0     111
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 125
14.9541%
Signals
-1    5424
 1    4389
Name: signals, dtype: int64
Signals change
0.0    9713
2.0      99
Name: signals, dtype: int64
-----------------------------